In [1]:
import pandas as pd
import ijson
import numpy as np
import json
import io
import os

In [20]:
data = pd.read_csv('LasVegas_tips_after2017.csv')

In [178]:
data.head(5)

,Unnamed: 0,business_id,text,date,attributes,categories,city,name,stars,review_count,class,tips_stars
0,8,KPgyqG3MyFUDK7GRbUg51A,Kelly Paun was so full of knowledge and was so...,2017-03-23,NaN,"Landmarks & Historical Buildings, Travel Servi...",Las Vegas,Pink Jeep Tours,4.5,136,1.0,Kelly Paun was so full of knowledge and was so...
1,20,JtNVcqioJhS8GZ-If30oHg,Make sure you sign up for Emerald club before ...,2018-03-29,NaN,"Automotive, Hotels & Travel, Car Rental",Las Vegas,National Car Rental,4.0,224,1.0,Make sure you sign up for Emerald club before ...
2,34,gh6__q2WXFuyN8gt6VAnWw,Very delicious food with very fast delivery!,2017-09-24,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...","Seafood, Restaurants, Steakhouses, Greek, Fren...",Las Vegas,Vila Algarve,4.5,189,1.0,Very delicious food with very fast delivery! 4.5
3,55,gh6__q2WXFuyN8gt6VAnWw,Amazing Portuguese food! Linguica (chirizo) is...,2017-12-11,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...","Seafood, Restaurants, Steakhouses, Greek, Fren...",Las Vegas,Vila Algarve,4.5,189,1.0,Amazing Portuguese food! Linguica (chirizo) is...
4,56,gh6__q2WXFuyN8gt6VAnWw,All the way from Porto portugal lost off love!...,2018-04-10,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...","Seafood, Restaurants, Steakhouses, Greek, Fren...",Las Vegas,Vila Algarve,4.5,189,1.0,All the way from Porto portugal lost off love!...


In [11]:
data.dtypes

Unnamed: 0        int64
business_id      object
text             object
date             object
attributes       object
categories       object
city             object
name             object
stars           float64
review_count      int64
dtype: object

## Define a categorical variable

In [12]:
data.index

RangeIndex(start=0, stop=65280, step=1)

In [22]:
stars = np.array(data['stars'])

In [23]:
##1.0 means good; 0.0 means not good, bad ...
stars[stars<4.0] = 0.0
stars[stars>=4.0] = 1.0
data['class'] = stars

In [177]:
data.head(15);

### A glimpse onto the statistics of the class variable

In [29]:
data['stars'].describe()

count    65280.000000
mean         3.908058
std          0.703868
min          1.000000
25%          3.500000
50%          4.000000
75%          4.500000
max          5.000000
Name: stars, dtype: float64

In [26]:
data['class'].describe()

count    65280.000000
mean         0.672733
std          0.469219
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: class, dtype: float64

#### Combine the tips column with the stars column

In [153]:
# Create a new column which combiens "text" and "stars"

# Turn stars into strings, add a space in front of each item

data['stars'] = data['stars'].map(str).apply(lambda x: ' ' + x);

data['tips_stars'] = data["text"] + data['stars']

In [156]:
data.iloc[98]['tips_stars']

"Park valet. It'll be the best $20 you spend because it gives you free valet parking nearly everywhere else (like Bellagio). Any MGM resort on the Strip is included, and there's many.  3.0"

#### Split the dataset into training set and test set 

In [32]:
from sklearn.model_selection import train_test_split

In [157]:
train_set, test_set = train_test_split(
...     data, test_size=0.33, random_state=42)

#### tf-idf

In [158]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer =  TfidfVectorizer(max_features=1200,stop_words='english',ngram_range=(1,3))

In [166]:
vectorizer.fit(data['tips_stars'].values.astype('U'))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1200, min_df=1,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [167]:
train_tfidf = vectorizer.transform(train_set['tips_stars'].values.astype('U'))

In [168]:
test_tfidf = vectorizer.transform(test_set['tips_stars'].values.astype('U'))

In [169]:
train_tfidf.shape

(43737, 1200)

In [170]:
test_tfidf.shape

(21543, 1200)

In [176]:
#print(vectorizer.get_feature_names())

In [130]:
#print(test_tfidf)

### K Means

In [110]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from sklearn_pandas import DataFrameMapper

## The output of K_Means is an array of labels. 

In [174]:
K = 16
KMeans_model = KMeans(n_clusters=K, init='k-means++', max_iter=100, n_init=1)
KMeans_model.fit(train_tfidf)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=16, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [191]:
KMeans_model.predict(train_tfidf)

array([ 3,  2,  2, ...,  2, 12, 13], dtype=int32)

In [175]:
order_centroids = KMeans_model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(K):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Cluster 0:
 don
 time
 come
 waste
 like
 just
 don forget
 order
 wait
 forget
Cluster 1:
 place
 great place
 great
 awesome
 nice
 good
 favorite
 place great
 nice place
 awesome place
Cluster 2:
 service
 delicious
 awesome
 food
 great
 free
 excellent
 just
 nice
 closed
Cluster 3:
 good
 food
 good food
 service
 food good
 good service
 really good
 really
 service good
 great
Cluster 4:
 love
 love love
 love food
 food
 great
 love love love
 good
 time
 coming
 service
Cluster 5:
 best
 town
 ve
 place
 best place
 food
 hands
 service
 pizza
 sushi
Cluster 6:
 amazing
 food
 amazing food
 service
 food amazing
 amazing service
 great
 place amazing
 place
 service amazing
Cluster 7:
 happy
 happy hour
 hour
 great
 6pm
 menu
 7pm
 drinks
 pm
 daily
Cluster 8:
 people
 nice
 nice people
 great
 great people
 place
 food
 good
 friendly
 service
Cluster 9:
 vegas
 las
 las vegas
 best
 favorite
 place
 place vegas
 food
 restaurant
 pizza
Cluster 10:
 love place
 love
 place

## Dimension Reducing: PCA

In [185]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

n_components = 50

#truncatedSVD 

In [186]:
svd = TruncatedSVD(n_components)

In [188]:
svd.fit_transform(train_tfidf)

array([[ 1.54680094e-01,  2.21561836e-01,  3.84991117e-01, ...,
        -1.52790677e-01, -1.10233875e-01,  7.27960195e-02],
       [ 3.89009403e-02,  2.52421114e-02,  5.06642381e-03, ...,
        -1.59692171e-02,  4.82814553e-02, -2.33014656e-02],
       [ 9.43317201e-03,  4.42481893e-03,  3.25021368e-04, ...,
         1.82142256e-03, -5.19702368e-03, -1.00569419e-02],
       ...,
       [ 4.55154522e-02,  9.14376852e-03, -5.32312840e-03, ...,
         7.64438501e-02,  2.71775208e-02,  3.22214719e-02],
       [ 1.22982433e-01,  1.73232219e-01, -7.08030983e-02, ...,
         6.34858218e-02,  8.44965219e-02, -3.91187608e-02],
       [ 5.13253029e-02,  2.03455494e-02, -1.80809492e-03, ...,
        -6.80150835e-02,  1.63058543e-02, -7.20319544e-03]])

In [194]:
train_tfidf

<43737x1200 sparse matrix of type '<class 'numpy.float64'>'
	with 231554 stored elements in Compressed Sparse Row format>

### Logistic Regression

In [192]:
from sklearn.linear_model import LogisticRegression

In [193]:
train_target = train_set["class"]

In [195]:
clf = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial').fit(train_tfidf,train_target)

In [196]:
clf.predict(train_tfidf)

array([1., 1., 1., ..., 1., 1., 0.])

In [197]:
clf.predict(test_tfidf)

array([1., 1., 1., ..., 1., 1., 0.])

In [198]:
clf.score(train_tfidf, train_target)

0.7227747673594439

In [199]:
clf.score(test_tfidf,test_set["class"])

0.700505964814557

In [200]:
train_set.count()

Unnamed: 0      43737
business_id     43737
text            43736
date            43737
attributes      42609
categories      43722
city            43737
name            43737
stars           43737
review_count    43737
class           43737
tips_stars      43736
dtype: int64

In [204]:
train_set[train_set["class"]==1.0].count()

Unnamed: 0      29505
business_id     29505
text            29504
date            29505
attributes      28946
categories      29501
city            29505
name            29505
stars           29505
review_count    29505
class           29505
tips_stars      29504
dtype: int64

In [205]:
29505/43737

0.6746004527059469

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = ["This little kitty came to play when I was eating at a restaurant.",
             "Merley has the best squooshy kitten belly.",
             "Google Translate app is incredible.",
             "If you open 100 tab in google you get a smiley face.",
             "Best cat photo I've ever taken.",
             "Climbing ninja cat.",
             "Impressed with google map feedback.",
             "Key promoter extension for Google Chrome."]

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)


 

In [84]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)



KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=2, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [85]:
print("Top terms per cluster:")


Top terms per cluster:


In [86]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print



Cluster 0:
 google
 ninja
 climbing
 app
 feedback
 impressed
 incredible
 translate
 map
 cat
Cluster 1:
 best
 ve
 photo
 taken
 belly
 merley
 kitten
 squooshy
 restaurant
 came


In [ ]:
print("\n")
print("Prediction")

Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print(prediction)